# **Ejercicio 4 (Maximización de Área con PSO)**

**Objetivo:** Determinar las dimensiones de una ventana, compuesta por un rectángulo con un semicírculo en la parte superior, que maximicen su área total (para permitir la mayor entrada de luz), utilizando una cantidad fija de 12 m de material para el marco y aplicando el algoritmo de Optimización por Enjambre de Partículas (PSO).

## **Configuración Inicial**

### **Instalar y Cargar Librerías**
Se importan las librerías necesarias. DEAP es el framework principal para implementar el algoritmo PSO.

In [1]:
# Si es necesario, instalar el paquete DEAP
# !pip install deap

In [2]:
# Importar paquetes
import matplotlib.pyplot as plt
from deap import base, creator, tools
import pandas as pd
import operator
import random
import numpy
import math

## **Planteamiento del Problema**

Se desea construir una ventana que consiste en un rectángulo con un semicírculo en la parte superior. Se dispone de 12 m de material para el marco.

* **Variables:**
    * `r`: Radio del semicírculo, que también es la mitad del ancho del rectángulo.
    * `h`: Altura de la parte rectangular.

* **Función Objetivo (Área):**
El área total es la suma del área del rectángulo y el área del semicírculo.

    $$ A = \text{Área del rectángulo} + \text{Área del semicírculo} = (2r)h + \frac{1}{2}\pi r^2 $$

* **Restricción (Perímetro):**
El material total para el marco es la suma de la base del rectángulo, las dos alturas y el arco del semicírculo.

    $$ \text{Perímetro Total} = (2r) + 2h + (\pi r) = 12 $$

    De esta restricción, podemos despejar la altura `h` en función de `r`:

    $$ 2h = 12 - 2r - \pi r $$
    $$ h = \frac{12 - 2r - \pi r}{2} = 6 - r - \frac{\pi}{2}r $$

    Sustituyendo `h` en la función de área, obtenemos la función objetivo dependiente de una sola variable, `r`:

    $$ \text{Maximizar } A(r) = 2r \left( 6 - r - \frac{\pi}{2}r \right) + \frac{1}{2}\pi r^2 $$
    $$ A(r) = 12r - 2r^2 - \pi r^2 + \frac{1}{2}\pi r^2 $$
    $$ A(r) = 12r - 2r^2 - \frac{1}{2}\pi r^2 = 12r - r^2 \left( 2 + \frac{\pi}{2} \right) $$

* **Restricciones de Dominio:**
    * `r > 0` (El radio debe ser positivo).
    * La restricción `h > 0` (la altura debe ser positiva) nos lleva a la siguiente lógica para encontrar el límite superior de `r`:

        1.  **Fórmula de la Altura:** Expresamos la altura en función de `r`.
            $$ h = 6 - r - \frac{\pi}{2}r $$

        2.  **Aplicar la Restricción:** Como la altura debe ser mayor que cero (`h > 0`).
            $$ 6 - r - \frac{\pi}{2}r > 0 $$

        3.  **Resolver la Desigualdad:**
            $$ 6 > r + \frac{\pi}{2}r $$
            $$ 6 > r \left( 1 + \frac{\pi}{2} \right) $$
            $$ \frac{6}{1 + \frac{\pi}{2}} > r $$

        4.  **Conclusión:**
            $$ r < \frac{12}{2 + \pi} \approx 2.333 $$

    Por lo tanto, el radio `r` de la base debe ser menor que aproximadamente 2.333 metros para que la ventana tenga una altura positiva.

### **Funciones del Problema**
* **`objective_function`**: Calcula el área de la ventana, que es el valor a maximizar.
* **`feasible`**: Verifica si una partícula (un valor de `r`) cumple con las restricciones de dominio.

In [3]:
# Función Objetivo
# Maximizar el área A(r) = 12*r - 2*r^2 - (pi/2)*r^2
def objective_function(individual):
    r = individual[0]
    # Se desglosa la fórmula para mayor claridad.
    area = 12*r - 2*(r**2) - (math.pi/2)*(r**2)
    return area,

In [4]:
# Restricción del Problema
# Verifica si la partícula es una solución factible (0 < r < 12 / (2 + pi)).
def feasible(individual):
    r = individual[0]
    if r < 0:
        return False
    # La restricción h > 0 implica r < 12 / (2 + pi)
    if r > 12 / (2 + math.pi):
      return False
    return True

## **Configuración del Algoritmo PSO**

### **1. Creación de Tipos con `creator`**
Se definen las estructuras para el **Fitness** y la **Partícula**.

* **`FitnessMax`**: Define un objetivo de maximización con `weights=(1.0,)`.
* **`Particle`**: Define la estructura de una partícula con sus atributos específicos para PSO: `speed` (velocidad), `smin`/`smax` (límites de velocidad) y `best` (mejor posición personal).

In [5]:
# Crear los tipos de Fitness y Partícula
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, smin=None, smax=None, best=None)

### **2. Funciones para Generar y Actualizar Partículas**
* **`generate`**: Inicializa una partícula con una posición y velocidad aleatorias.
* **`updateParticle`**: Implementa la lógica de movimiento de PSO, actualizando la velocidad y posición de la partícula basándose en su mejor posición y la del enjambre.

In [6]:
# Función para generar una partícula con posición y velocidad iniciales aleatorias.
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

In [7]:
# Función para actualizar la velocidad y posición de una partícula.
def updateParticle(part, best, phi1, phi2):
    # Componente cognitivo (hacia la mejor posición personal)
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    # Componente social (hacia la mejor posición global)
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    # Actualizar velocidad y aplicar límites
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if abs(speed) < part.smin:
            part.speed[i] = math.copysign(part.smin, speed)
        elif abs(speed) > part.smax:
            part.speed[i] = math.copysign(part.smax, speed)
    # Actualizar posición
    part[:] = list(map(operator.add, part, part.speed))

### **3. Creación de la `Toolbox`**
Se registran las funciones y operadores en la `Toolbox` para su uso en el algoritmo.

In [8]:
# Crear la caja de herramientas (Toolbox)
toolbox = base.Toolbox()

# Registrar la función para generar partículas.
# size=1 (solo la variable r), pmin/pmax (límites de posición), smin/smax (límites de velocidad).
# pmax se establece en 3.0, un valor razonable por encima del límite teórico (~2.33).
toolbox.register("particle", generate, size=1, pmin=0, pmax=3.0, smin=-0.5, smax=0.5)

# Registrar la función para generar la población.
toolbox.register("population", tools.initRepeat, list, toolbox.particle)

# Registrar la función para actualizar las partículas.
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)

# Registrar la función de evaluación con penalización para soluciones no factibles.
toolbox.register("evaluate", objective_function)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, -100000)) # Penalización alta si no es factible

### **4. Definición de Parámetros y Estadísticas**
Se configuran los parámetros del algoritmo y el registro de estadísticas.

In [9]:
# Parámetros del algoritmo PSO
initial_population = 100    # Número de partículas
num_ite = 100               # Número de iteraciones
best = None                 # Mejor partícula global

In [10]:
# Creación de la población inicial
pop = toolbox.population(n=initial_population)

In [11]:
# Configuración de las estadísticas a monitorear
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

In [12]:
# Configuración del Logbook para guardar el historial
logbook = tools.Logbook()
logbook.header = ["gen", "evals"] + stats.fields

## **Ejecución del Algoritmo PSO**
Se ejecuta el bucle principal de optimización.

In [13]:
# Bucle principal de la optimización PSO
for iteration in range(num_ite):
    for part in pop:
        # Evaluar fitness
        part.fitness.values = toolbox.evaluate(part)
        # Actualizar mejor posición personal (pbest)
        if not part.best or part.best.fitness < part.fitness:
            part.best = creator.Particle(part)
            part.best.fitness.values = part.fitness.values
        # Actualizar mejor posición global (gbest)
        if not best or best.fitness < part.fitness:
            best = creator.Particle(part)
            best.fitness.values = part.fitness.values
    # Actualizar velocidad y posición de todas las partículas
    for part in pop:
        toolbox.update(part, best)

    # Guardar y mostrar las estadísticas de la iteración
    logbook.record(gen=iteration, evals=len(pop), **stats.compile(pop))
    print(logbook.stream)

gen	evals	avg     	std    	min    	max    
0  	100  	-23994.4	42711.5	-100000	10.0816
1  	100  	-10992  	31291.8	-100000	10.0808
2  	100  	-2990.57	17060.4	-100000	10.0814
3  	100  	-1990.5 	14001.4	-100000	10.0817
4  	100  	-990.424	9950.84	-100000	10.0818
5  	100  	-2990.59	17060.4	-100000	10.081 
6  	100  	9.72226 	0.325387	8.88927	10.0818
7  	100  	-990.32 	9950.85 	-100000	10.0817
8  	100  	-990.345	9950.84 	-100000	10.0818
9  	100  	-1990.46	14001.4 	-100000	10.0818
10 	100  	9.77477 	0.316502	8.74694	10.0818
11 	100  	9.72481 	0.411423	7.62109	10.0818
12 	100  	9.84772 	0.28719 	9.1268 	10.0818
13 	100  	9.71123 	0.34265 	9.05434	10.0818
14 	100  	-1990.33	14001.4 	-100000	10.0817
15 	100  	-990.36 	9950.84 	-100000	10.0818
16 	100  	-990.27 	9950.85 	-100000	10.0818
17 	100  	-990.354	9950.84 	-100000	10.0818
18 	100  	9.76989 	0.370298	8.58318	10.0818
19 	100  	9.7975  	0.31068 	8.77056	10.0818
20 	100  	9.82616 	0.304141	8.85622	10.0818
21 	100  	-990.335	9950.85 	-100000	10.

## **Resultados Finales**
Se muestran las dimensiones y el área máxima de la ventana encontrados por el algoritmo.

In [14]:
# Imprimir la mejor solución encontrada
r_optimo = best[0]
h_optima = 6 - r_optimo - (math.pi / 2) * r_optimo

print(f"Radio del semicírculo (r): {r_optimo:.4f} m")
print(f"Altura de la parte rectangular (h): {h_optima:.4f} m")
print(f"Ancho total de la ventana (2r): {2*r_optimo:.4f} m")

Radio del semicírculo (r): 1.6803 m
Altura de la parte rectangular (h): 1.6804 m
Ancho total de la ventana (2r): 3.3605 m


In [15]:
# Evaluar la mejor solución
max_area = objective_function(best)[0]
perimetro_usado = 2 * r_optimo + 2 * h_optima + math.pi * r_optimo

print(f"Área Máxima: {max_area:.4f} m²")
print(f"Material del marco utilizado: {perimetro_usado:.4f} m")

Área Máxima: 10.0818 m²
Material del marco utilizado: 12.0000 m


In [16]:
# Verificar si la solución es factible
print(f"Cumple restricciones: {feasible(best)}")

Cumple restricciones: True
